In [2]:
import os
import math
import numpy as np
import pickle as pkl
import re

# Stereological

### Poisson rate - lambda

In [3]:
# bias / coverage table
observations = [100, 500, 1000, 5000]
base_dir = "../res/stereological/"

simulation_functions = [
    lambda n: n,
    lambda n: int(n * math.log(n)),
    lambda n: int(n ** (3/2)),
    lambda n: n ** 2
]

num_seeds = 101
true_param = 100.0

latex_table = "\\begin{table}[h]\n\\centering\n\\begin{tabular}{@{}l l l l l @{} }\\toprule\n"
latex_table += " & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\\\ \n\\hline\n"

for n_obs in observations:
    latex_table += f"$n={n_obs}$ &"
    for sim_func in simulation_functions:
        n_sims = sim_func(n_obs)
        if n_sims == 25_000_000:
            n_sims = 25_000_00
        mean_biases = []
        median_biases = []
        coverage_80 = []
        coverage_90 = []
        coverage_95 = []
        for i in range(num_seeds):
            dir_path = os.path.join(base_dir, f"npe_n_obs_{n_obs}_n_sims_{n_sims}_seed_{str(i)}/")
            if os.path.exists(dir_path):
                # true_samples_path = os.path.join(dir_path, "true_posterior_samples.pkl")
                npe_samples_path = os.path.join(dir_path, "posterior_samples.pkl")
                coverage_path = os.path.join(dir_path, "estimated_coverage.npy")
                try:
                    with open(npe_samples_path, 'rb') as file:
                        posterior_samples = pkl.load(file)
                        seed_mean_bias = np.mean(posterior_samples, axis=0)[0] - true_param
                        seed_median_bias = np.median(posterior_samples, axis=0)[0] - true_param
                        mean_biases.append(seed_mean_bias)
                        median_biases.append(seed_median_bias)

                        coverage = np.load(coverage_path)

                        coverage = np.array(coverage)
                        coverage_80.append(coverage[0][0])
                        coverage_90.append(coverage[0][1])
                        coverage_95.append(coverage[0][2])
                except FileNotFoundError:
                    continue

        # Calculate mean bias over all seeds if biases were collected
        if coverage_80:
            mean_bias = np.mean(np.abs(mean_biases))
            std_dev_mean_bias = np.std(mean_biases)
            median_bias = np.mean(np.abs(median_biases))
            std_dev_median_bias = np.std(median_biases)
            coverage_80 = np.mean(coverage_80)
            coverage_90 = np.mean(coverage_90)
            coverage_95 = np.mean(coverage_95)
            latex_table += f"{coverage_80:.2f}/{coverage_90:.2f}/{coverage_95:.2f} &"
        else:
            latex_table += f"-"
    latex_table += " \\\\ \n"
# Close the table
latex_table += "\\bottomrule"
latex_table += "\\end{tabular}\n\\end{table}"

# Print the LaTeX table string
print(latex_table)

\begin{table}[h]
\centering
\begin{tabular}{@{}l l l l l @{} }\toprule
 & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\ 
\hline
$n=100$ &1.00/1.00/1.00 &0.93/0.99/1.00 &0.81/0.94/0.99 &0.81/0.91/0.96 & \\ 
$n=500$ &0.97/1.00/1.00 &0.87/0.96/0.99 &0.83/0.93/0.97 &0.82/0.91/0.96 & \\ 
$n=1000$ &0.97/1.00/1.00 &0.87/0.95/0.99 &0.84/0.93/0.98 &0.80/0.90/0.95 & \\ 
$n=5000$ &0.93/0.98/1.00 &0.89/0.96/0.99 &0.86/0.95/0.98 &0.81/0.92/0.96 & \\ 
\bottomrule\end{tabular}
\end{table}


### Pareto Scale - xi

In [10]:
# bias / coverage table
observations = [100, 500, 1000, 5000]
base_dir = "../res/stereological/"

simulation_functions = [
    lambda n: n,
    lambda n: int(n * math.log(n)),
    lambda n: int(n ** (3/2)),
    lambda n: n ** 2
]

num_seeds = 101
true_param = 2.0

latex_table = "\\begin{table}[h]\n\\centering\n\\begin{tabular}{@{}l l l l l @{} }\\toprule\n"
latex_table += " & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\\\ \n\\hline\n"

for n_obs in observations:
    latex_table += f"$n={n_obs}$ &"
    for sim_func in simulation_functions:
        n_sims = sim_func(n_obs)
        mean_biases = []
        median_biases = []
        coverage_80 = []
        coverage_90 = []
        coverage_95 = []
        for i in range(num_seeds):
            dir_path = os.path.join(base_dir, f"npe_n_obs_{n_obs}_n_sims_{n_sims}_seed_{str(i)}/")
            if os.path.exists(dir_path):
                # true_samples_path = os.path.join(dir_path, "true_posterior_samples.pkl")
                npe_samples_path = os.path.join(dir_path, "posterior_samples.pkl")
                coverage_path = os.path.join(dir_path, "estimated_coverage.npy")
                try:
                    with open(npe_samples_path, 'rb') as file:
                        posterior_samples = pkl.load(file)
                        seed_mean_bias = np.mean(posterior_samples, axis=0)[1] - true_param
                        seed_median_bias = np.median(posterior_samples, axis=0)[1] - true_param
                        mean_biases.append(seed_mean_bias)
                        median_biases.append(seed_median_bias)

                        coverage = np.load(coverage_path)

                        coverage = np.array(coverage)
                        coverage_80.append(coverage[1][0])
                        coverage_90.append(coverage[1][1])
                        coverage_95.append(coverage[1][2])
                except FileNotFoundError:
                    continue

        # Calculate mean bias over all seeds if biases were collected
        if mean_biases:
            mean_bias = np.mean(np.abs(mean_biases))
            std_dev_mean_bias = np.std(mean_biases)
            median_bias = np.mean(np.abs(median_biases))
            std_dev_median_bias = np.std(median_biases)
            coverage_80 = np.mean(coverage_80)
            coverage_90 = np.mean(coverage_90)
            coverage_95 = np.mean(coverage_95)
            latex_table += f"{coverage_80:.2f}/{coverage_90:.2f}/{coverage_95:.2f} &"
        else:
            latex_table += f"-"
    latex_table += " \\\\ \n"
# Close the table
latex_table += "\\bottomrule"
latex_table += "\\end{tabular}\n\\end{table}"

# Print the LaTeX table string
print(latex_table)

\begin{table}[h]
\centering
\begin{tabular}{@{}l l l l l @{} }\toprule
 & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\ 
\hline
$n=100$ &1.00/1.00/1.00 &1.00/1.00/1.00 &0.98/1.00/1.00 &0.88/0.96/0.99 & \\ 
$n=500$ &1.00/1.00/1.00 &0.96/0.98/0.99 &0.90/0.97/0.99 &0.81/0.91/0.96 & \\ 
$n=1000$ &0.99/1.00/1.00 &0.95/0.99/1.00 &0.85/0.94/0.98 &0.82/0.92/0.96 & \\ 
$n=5000$ &0.97/0.99/1.00 &0.90/0.97/0.99 &0.81/0.91/0.96 &0.84/0.94/0.99 & \\ 
\bottomrule\end{tabular}
\end{table}


### Pareto shape - sigma

In [11]:
# bias / coverage table
observations = [100, 500, 1000, 5000]
base_dir = "../res/stereological/"

simulation_functions = [
    lambda n: n,
    lambda n: int(n * math.log(n)),
    lambda n: int(n ** (3/2)),
    lambda n: n ** 2
]

num_seeds = 101
true_param = -0.1

latex_table = "\\begin{table}[h]\n\\centering\n\\begin{tabular}{@{}l l l l l @{} }\\toprule\n"
latex_table += " & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\\\ \n\\hline\n"

for n_obs in observations:
    latex_table += f"$n={n_obs}$ & "
    for sim_func in simulation_functions:
        n_sims = sim_func(n_obs)
        mean_biases = []
        median_biases = []
        coverage_80 = []
        coverage_90 = []
        coverage_95 = []
        for i in range(num_seeds):
            dir_path = os.path.join(base_dir, f"npe_n_obs_{n_obs}_n_sims_{n_sims}_seed_{str(i)}/")
            if os.path.exists(dir_path):
                # true_samples_path = os.path.join(dir_path, "true_posterior_samples.pkl")
                npe_samples_path = os.path.join(dir_path, "posterior_samples.pkl")
                coverage_path = os.path.join(dir_path, "estimated_coverage.npy")
                try:
                    with open(npe_samples_path, 'rb') as file:
                        posterior_samples = pkl.load(file)
                        seed_mean_bias = np.mean(posterior_samples, axis=0)[2] - true_param
                        seed_median_bias = np.median(posterior_samples, axis=0)[2] - true_param
                        mean_biases.append(seed_mean_bias)
                        median_biases.append(seed_median_bias)

                        coverage = np.load(coverage_path)

                        coverage = np.array(coverage)
                        coverage_80.append(coverage[2][0])
                        coverage_90.append(coverage[2][1])
                        coverage_95.append(coverage[2][2])
                except FileNotFoundError:
                    continue

        # Calculate mean bias over all seeds if biases were collected
        if mean_biases:
            mean_bias = np.mean(np.abs(mean_biases))
            std_dev_mean_bias = np.std(mean_biases)
            median_bias = np.mean(np.abs(median_biases))
            std_dev_median_bias = np.std(median_biases)
            coverage_80 = np.mean(coverage_80)
            coverage_90 = np.mean(coverage_90)
            coverage_95 = np.mean(coverage_95)
            latex_table += f"{coverage_80:.2f}/{coverage_90:.2f}/{coverage_95:.2f} &"
        else:
            latex_table += f"-"
    latex_table += " \\\\ \n"
# Close the table
latex_table += "\\bottomrule"
latex_table += "\\end{tabular}\n\\end{table}"

# Print the LaTeX table string
print(latex_table)

\begin{table}[h]
\centering
\begin{tabular}{@{}l l l l l @{} }\toprule
 & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\ 
\hline
$n=100$ & 0.26/0.49/0.79 &0.34/0.67/0.95 &0.83/0.88/0.98 &0.93/0.98/1.00 & \\ 
$n=500$ & 0.30/0.68/0.96 &1.00/0.99/1.00 &0.94/0.98/1.00 &0.83/0.93/0.97 & \\ 
$n=1000$ & 0.69/0.84/0.99 &0.99/0.99/1.00 &0.90/0.97/0.99 &0.84/0.93/0.97 & \\ 
$n=5000$ & 0.98/0.99/1.00 &0.91/0.97/0.99 &0.89/0.97/0.99 &0.92/0.97/0.99 & \\ 
\bottomrule\end{tabular}
\end{table}


# gnk

### A

In [ ]:
# bias / coverage table
observations = [100, 500, 1000, 5000]
base_dir = "../res/gnk/"

simulation_functions = [
    lambda n: n,
    lambda n: int(n * math.log(n)),
    lambda n: int(n ** (3/2)),
    lambda n: n ** 2
]

num_seeds = 101
true_param = 3.0

latex_table = "\\begin{table}[h]\n\\centering\n\\begin{tabular}{@{}l l l l l @{} }\\toprule\n"
latex_table += " & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\\\ \n\\hline\n"

for n_obs in observations:
    latex_table += f"$n={n_obs}$ &"
    for sim_func in simulation_functions:
        n_sims = sim_func(n_obs)
        mean_biases = []
        median_biases = []
        coverage_80 = []
        coverage_90 = []
        coverage_95 = []
        for i in range(num_seeds):
            dir_path = os.path.join(base_dir, f"npe_n_obs_{n_obs}_n_sims_{n_sims}_seed_{str(i)}/")
            if os.path.exists(dir_path):
                # true_samples_path = os.path.join(dir_path, "true_posterior_samples.pkl")
                npe_samples_path = os.path.join(dir_path, "posterior_samples.pkl")
                coverage_path = os.path.join(dir_path, "estimated_coverage.npy")
                try:
                    with open(npe_samples_path, 'rb') as file:
                        posterior_samples = pkl.load(file)
                        seed_mean_bias = np.mean(posterior_samples, axis=0)[0] - true_param
                        seed_median_bias = np.median(posterior_samples, axis=0)[0] - true_param
                        mean_biases.append(seed_mean_bias)
                        median_biases.append(seed_median_bias)

                        coverage = np.load(coverage_path)

                        coverage = np.array(coverage)
                        coverage_80.append(coverage[0][0])
                        coverage_90.append(coverage[0][1])
                        coverage_95.append(coverage[0][2])
                except FileNotFoundError:
                    continue

        # Calculate mean bias over all seeds if biases were collected
        if mean_biases:
            mean_bias = np.mean(np.abs(mean_biases))
            std_dev_mean_bias = np.std(mean_biases)
            median_bias = np.mean(np.abs(median_biases))
            std_dev_median_bias = np.std(median_biases)
            coverage_80 = np.mean(coverage_80)
            coverage_90 = np.mean(coverage_90)
            coverage_95 = np.mean(coverage_95)
            latex_table += f"{coverage_80:.2f}/{coverage_90:.2f}/{coverage_95:.2f} &"
        else:
            latex_table += f"-"
    latex_table += " \\\\ \n"
# Close the table
latex_table += "\\bottomrule"
latex_table += "\\end{tabular}\n\\end{table}"

# Print the LaTeX table string
print(latex_table)

\begin{table}[h]
\centering
\begin{tabular}{@{}l l l l l @{} }\toprule
 & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\ 
\hline
$n=100$ &0.99/1.00/1.00 &1.00/1.00/1.00 &0.99/1.00/1.00 &0.85/0.94/0.98 & \\ 
$n=500$ &1.00/1.00/1.00 &0.89/0.97/1.00 &0.78/0.90/0.95 &0.76/0.87/0.93 & \\ 
$n=1000$ &0.99/1.00/1.00 &0.83/0.92/0.97 &0.78/0.88/0.94 &0.80/0.90/0.94 & \\ 
$n=5000$ &0.94/0.99/1.00 &0.83/0.93/0.97 &0.78/0.89/0.94 &0.80/0.90/0.95 & \\ 
\bottomrule\end{tabular}
\end{table}


### B

In [ ]:
# bias / coverage table
observations = [100, 500, 1000, 5000]
base_dir = "../res/gnk/"

simulation_functions = [
    lambda n: n,
    lambda n: int(n * math.log(n)),
    lambda n: int(n ** (3/2)),
    lambda n: n ** 2
]

num_seeds = 101
true_param = 1.0

latex_table = "\\begin{table}[h]\n\\centering\n\\begin{tabular}{@{}l l l l l @{} }\\toprule\n"
latex_table += " & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\\\ \n\\hline\n"

for n_obs in observations:
    latex_table += f"$n={n_obs}$ &"
    for sim_func in simulation_functions:
        n_sims = sim_func(n_obs)
        mean_biases = []
        median_biases = []
        coverage_80 = []
        coverage_90 = []
        coverage_95 = []
        for i in range(num_seeds):
            dir_path = os.path.join(base_dir, f"npe_n_obs_{n_obs}_n_sims_{n_sims}_seed_{str(i)}/")
            if os.path.exists(dir_path):
                # true_samples_path = os.path.join(dir_path, "true_posterior_samples.pkl")
                npe_samples_path = os.path.join(dir_path, "posterior_samples.pkl")
                coverage_path = os.path.join(dir_path, "estimated_coverage.npy")
                try:
                    with open(npe_samples_path, 'rb') as file:
                        posterior_samples = pkl.load(file)
                        seed_mean_bias = np.mean(posterior_samples, axis=0)[1] - true_param
                        seed_median_bias = np.median(posterior_samples, axis=0)[1] - true_param
                        mean_biases.append(seed_mean_bias)
                        median_biases.append(seed_median_bias)

                        coverage = np.load(coverage_path)

                        coverage = np.array(coverage)
                        coverage_80.append(coverage[1][0])
                        coverage_90.append(coverage[1][1])
                        coverage_95.append(coverage[1][2])
                except FileNotFoundError:
                    continue

        # Calculate mean bias over all seeds if biases were collected
        if mean_biases:
            mean_bias = np.mean(np.abs(mean_biases))
            std_dev_mean_bias = np.std(mean_biases)
            median_bias = np.mean(np.abs(median_biases))
            std_dev_median_bias = np.std(median_biases)
            coverage_80 = np.mean(coverage_80)
            coverage_90 = np.mean(coverage_90)
            coverage_95 = np.mean(coverage_95)
            latex_table += f"{coverage_80:.2f}/{coverage_90:.2f}/{coverage_95:.2f} &"
        else:
            latex_table += f"-"
    latex_table += " \\\\ \n"
# Close the table
latex_table += "\\bottomrule"
latex_table += "\\end{tabular}\n\\end{table}"

# Print the LaTeX table string
print(latex_table)

\begin{table}[h]
\centering
\begin{tabular}{@{}l l l l l @{} }\toprule
 & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\ 
\hline
$n=100$ &0.96/1.00/1.00 &1.00/1.00/1.00 &1.00/1.00/1.00 &0.97/1.00/1.00 & \\ 
$n=500$ &1.00/1.00/1.00 &0.97/1.00/1.00 &0.91/0.99/1.00 &0.84/0.92/0.97 & \\ 
$n=1000$ &0.95/0.99/1.00 &0.91/0.98/1.00 &0.89/0.97/0.99 &0.84/0.94/0.97 & \\ 
$n=5000$ &0.94/0.99/1.00 &0.90/0.97/0.99 &0.87/0.95/0.98 &0.84/0.93/0.97 & \\ 
\bottomrule\end{tabular}
\end{table}


### g

In [17]:
# bias / coverage table
observations = [100, 500, 1000, 5000]
base_dir = "../res/gnk/"

simulation_functions = [
    lambda n: n,
    lambda n: int(n * math.log(n)),
    lambda n: int(n ** (3/2)),
    lambda n: n ** 2
]

num_seeds = 101
true_param = 2.0
count = 0

latex_table = "\\begin{table}[h]\n\\centering\n\\begin{tabular}{@{}l l l l l @{} }\\toprule\n"
latex_table += " & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\\\ \n\\hline\n"

for n_obs in observations:
    latex_table += f"$n={n_obs}$ &"
    for sim_func in simulation_functions:
        n_sims = sim_func(n_obs)
        mean_biases = []
        median_biases = []
        coverage_80 = []
        coverage_90 = []
        coverage_95 = []
        for i in range(num_seeds):
            dir_path = os.path.join(base_dir, f"npe_n_obs_{n_obs}_n_sims_{n_sims}_seed_{str(i)}/")
            if os.path.exists(dir_path):
                # true_samples_path = os.path.join(dir_path, "true_posterior_samples.pkl")
                npe_samples_path = os.path.join(dir_path, "posterior_samples.pkl")
                coverage_path = os.path.join(dir_path, "estimated_coverage.npy")
                try:
                    with open(npe_samples_path, 'rb') as file:
                        posterior_samples = pkl.load(file)
                        seed_mean_bias = np.mean(posterior_samples, axis=0)[2] - true_param
                        seed_median_bias = np.median(posterior_samples, axis=0)[2] - true_param
                        mean_biases.append(seed_mean_bias)
                        median_biases.append(seed_median_bias)

                        coverage = np.load(coverage_path)

                        coverage = np.array(coverage)
                        coverage_80.append(coverage[2][0])
                        coverage_90.append(coverage[2][1])
                        coverage_95.append(coverage[2][2])
                except FileNotFoundError:
                    # print(f"n_obs: {n_obs}, n_sims: {n_sims}, seed: {i}")
                    continue

        # Calculate mean bias over all seeds if biases were collected
        print(coverage_80)
        if mean_biases:
            mean_bias = np.mean(np.abs(mean_biases))
            std_dev_mean_bias = np.std(mean_biases)
            median_bias = np.mean(np.abs(median_biases))
            std_dev_median_bias = np.std(median_biases)
            coverage_80 = np.mean(coverage_80)
            coverage_90 = np.mean(coverage_90)
            coverage_95 = np.mean(coverage_95)
            latex_table += f"{coverage_80:.2f}/{coverage_90:.2f}/{coverage_95:.2f} &"
        else:
            latex_table += f"-"
    latex_table += " \\\\ \n"
# Close the table
latex_table += "\\bottomrule"
latex_table += "\\end{tabular}\n\\end{table}"

# Print the LaTeX table string
print(latex_table)
# print(count)

[1.0, 1.0, 1.0, 0.58, 1.0, 1.0, 1.0, 1.0, 0.12, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.91, 0.98, 1.0, 0.0, 0.14, 1.0, 1.0, 0.63, 1.0, 1.0, 1.0, 0.62, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.94, 1.0, 0.38, 0.65, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.7, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.01, 1.0, 0.21, 1.0, 1.0, 0.0, 1.0, 0.13, 0.0, 1.0, 1.0, 0.0, 0.34, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 0.22, 0.22, 0.0, 0.96, 0.0, 0.0, 1.0, 0.03, 0.0, 1.0, 0.0, 0.96, 0.05, 0.0, 1.0, 0.0, 1.0, 0.81, 1.0, 0.0, 0.0, 1.0, 0.0, 0.96, 0.01, 0.79, 0.2, 0.0, 0.55, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.41, 1.0, 0.0, 1.0, 0.0, 0.65, 0.11, 0.98, 0.87, 0.31, 0.0, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.99, 0.0, 0.92, 0.0, 0.0, 0.0, 0.21, 0.09, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.17, 0.68, 1.0, 0.02, 1.0, 0.0, 0.0, 0.0, 0.99, 1.0, 0.0, 1.0, 0.98, 0.0, 0.0, 0.0, 0.01, 0

In [37]:
coverage_path

'../res/gnk/npe_n_obs_5000_n_sims_25000000_seed_20/estimated_coverage.npy'

In [71]:
coverage_path = '../res/gnk/npe_n_obs_5000_n_sims_353553_seed_15/estimated_coverage.npy'

with open(npe_samples_path, 'rb') as file:
    coverage = np.load(coverage_path)

    coverage = np.array(coverage)

# coverage[2][0]
# coverage[1][2]

0.95

### k

In [18]:
# bias / coverage table
observations = [100, 500, 1000, 5000]
base_dir = "../res/gnk/"

simulation_functions = [
    lambda n: n,
    lambda n: int(n * math.log(n)),
    lambda n: int(n ** (3/2)),
    lambda n: n ** 2
]

num_seeds = 101
true_param = 0.5

latex_table = "\\begin{table}[h]\n\\centering\n\\begin{tabular}{@{}l l l l l @{} }\\toprule\n"
latex_table += " & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\\\ \n\\hline\n"

for n_obs in observations:
    latex_table += f"$n={n_obs}$ &"
    for sim_func in simulation_functions:
        n_sims = sim_func(n_obs)
        mean_biases = []
        median_biases = []
        coverage_80 = []
        coverage_90 = []
        coverage_95 = []
        for i in range(num_seeds):
            dir_path = os.path.join(base_dir, f"npe_n_obs_{n_obs}_n_sims_{n_sims}_seed_{str(i)}/")
            if os.path.exists(dir_path):
                # true_samples_path = os.path.join(dir_path, "true_posterior_samples.pkl")
                npe_samples_path = os.path.join(dir_path, "posterior_samples.pkl")
                coverage_path = os.path.join(dir_path, "estimated_coverage.npy")
                try:
                    with open(npe_samples_path, 'rb') as file:
                        posterior_samples = pkl.load(file)
                        seed_mean_bias = np.mean(posterior_samples, axis=0)[3] - true_param
                        seed_median_bias = np.median(posterior_samples, axis=0)[3] - true_param
                        mean_biases.append(seed_mean_bias)
                        median_biases.append(seed_median_bias)

                        coverage = np.load(coverage_path)

                        coverage = np.array(coverage)
                        coverage_80.append(coverage[3][0])
                        coverage_90.append(coverage[3][1])
                        coverage_95.append(coverage[3][2])
                except FileNotFoundError:
                    continue

        # Calculate mean bias over all seeds if biases were collected
        if mean_biases:
            mean_bias = np.mean(np.abs(mean_biases))
            std_dev_mean_bias = np.std(mean_biases)
            median_bias = np.mean(np.abs(median_biases))
            std_dev_median_bias = np.std(median_biases)
            coverage_80 = np.mean(coverage_80)
            coverage_90 = np.mean(coverage_90)
            coverage_95 = np.mean(coverage_95)
            latex_table += f"{coverage_80:.2f}/{coverage_90:.2f}/{coverage_95:.2f} &"
        else:
            latex_table += f"-"
    latex_table += " \\\\ \n"
# Close the table
latex_table += "\\bottomrule"
latex_table += "\\end{tabular}\n\\end{table}"

# Print the LaTeX table string
print(latex_table)

\begin{table}[h]
\centering
\begin{tabular}{@{}l l l l l @{} }\toprule
 & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\ 
\hline
$n=100$ &0.80/0.89/0.94 &0.98/1.00/1.00 &1.00/1.00/1.00 &1.00/1.00/1.00 & \\ 
$n=500$ &0.92/0.99/1.00 &0.99/1.00/1.00 &0.97/1.00/1.00 &0.92/0.98/1.00 & \\ 
$n=1000$ &0.97/1.00/1.00 &0.97/1.00/1.00 &0.97/1.00/1.00 &0.87/0.96/0.99 & \\ 
$n=5000$ &0.96/1.00/1.00 &0.98/1.00/1.00 &0.94/0.99/1.00 &0.87/0.94/0.97 & \\ 
\bottomrule\end{tabular}
\end{table}


## gnk preconditioning - g

In [ ]:
# bias / coverage table
observations = [100, 500, 1000, 5000]
base_dir = "../res/gnk_precondition/"

simulation_functions = [
    lambda n: n,
    lambda n: int(n * math.log(n)),
    lambda n: int(n ** (3/2)),
    lambda n: n ** 2
]

num_seeds = 101
true_param = 2.0

latex_table = "\\begin{table}[h]\n\\centering\n\\begin{tabular}{@{}l l l l l @{} }\\toprule\n"
latex_table += " & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\\\ \n\\hline\n"

for n_obs in observations:
    latex_table += f"$n={n_obs}$ &"
    for sim_func in simulation_functions:
        n_sims = sim_func(n_obs)
        mean_biases = []
        median_biases = []
        coverage_80 = []
        coverage_90 = []
        coverage_95 = []
        for i in range(num_seeds):
            dir_path = os.path.join(base_dir, f"npe_n_obs_{n_obs}_n_sims_{n_sims}_seed_{str(i)}/")
            if os.path.exists(dir_path):
                # true_samples_path = os.path.join(dir_path, "true_posterior_samples.pkl")
                npe_samples_path = os.path.join(dir_path, "posterior_samples.pkl")
                coverage_path = os.path.join(dir_path, "estimated_coverage.npy")
                try:
                    with open(npe_samples_path, 'rb') as file:
                        posterior_samples = pkl.load(file)
                        seed_mean_bias = np.mean(posterior_samples, axis=0)[2] - true_param
                        seed_median_bias = np.median(posterior_samples, axis=0)[2] - true_param
                        mean_biases.append(seed_mean_bias)
                        median_biases.append(seed_median_bias)

                        coverage = np.load(coverage_path)

                        coverage = np.array(coverage)
                        coverage_80.append(coverage[2][0])
                        coverage_90.append(coverage[2][1])
                        coverage_95.append(coverage[2][2])
                except FileNotFoundError:
                    continue

        # Calculate mean bias over all seeds if biases were collected
        # print(coverage_80)
        if mean_biases:
            mean_bias = np.mean(np.abs(mean_biases))
            std_dev_mean_bias = np.std(mean_biases)
            median_bias = np.mean(np.abs(median_biases))
            std_dev_median_bias = np.std(median_biases)
            coverage_80 = np.mean(coverage_80)
            coverage_90 = np.mean(coverage_90)
            coverage_95 = np.mean(coverage_95)
            latex_table += f"{coverage_80:.2f}/{coverage_90:.2f}/{coverage_95:.2f} &"
        else:
            latex_table += f"-"
    latex_table += " \\\\ \n"
# Close the table
latex_table += "\\bottomrule"
latex_table += "\\end{tabular}\n\\end{table}"

# Print the LaTeX table string
print(latex_table)

\begin{table}[h]
\centering
\begin{tabular}{@{}l l l l l @{} }\toprule
 & $N=n$ &  $N=n\log(n)$ & $N=n^{3/2}$ & $N=n^2$ \\ 
\hline
$n=100$ &0.62/0.81/0.90 &0.57/0.94/1.00 &0.71/0.87/1.00 &0.76/0.98/1.00 & \\ 
$n=500$ &0.49/0.78/0.95 &0.78/0.92/1.00 &0.95/0.98/1.00 &0.89/0.96/0.99 & \\ 
$n=1000$ &0.64/0.95/1.00 &0.81/1.00/1.00 &1.00/1.00/1.00 &0.80/0.91/0.97 & \\ 
$n=5000$ &0.92/0.97/1.00 &1.00/1.00/1.00 &0.89/0.96/0.99 &0.78/0.89/0.95 & \\ 
\bottomrule\end{tabular}
\end{table}
